## 04.P Geospatial Data Problem Set – Summarizing Airborne Snow Observatory Data

__Background:__ For your graduate research, you are using lidar-derived snow water equivalent (SWE) estimates to assimilate into a physics based hydrology model to make streamflow predictions. In thinking through your hypotheses portions of the landscape where SWE data will have the most impact, you are reminded of a figure you saw in a presentation at a recent meeting. The figure shows how the distribution of SWE and land area across a range of elevation interact to control the distribution of water storage in the landscape. The figure looked like the following:

<img src='../img/aso_swe.png' alt='Upper Colorado ASO Analysis' width='750' style='display: block; margin: 0 auto'/>

You already have grids of estimated SWE from the Airborne Snow Observatory (ASO) for 4 separate dates at a 50 m spatial resolution, and a coregistered digital elevation model of the study area – the East River in the Colorado Rocky Mountains.  

__Task:__ Use the skills you've developed and the example notebooks you've examined in this module to create an image like the above. The ASO and coregistered DEM can be found in [this Google Drive folder](https://drive.google.com/drive/u/0/folders/1SkjOWPPJe5N25arCW6qbf2ZJWQsrUAAp). Use any of the four ASO files, which corresponds to 2 different dates in 2018 or 2019, or write your code generically to plot any or all dates. You should only need `rasterio`, `numpy`, and `matplotlib` to create this figure.    

In [ ]:
import os
import numpy as np
import rasterio as rio
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
# need to bring this function in
# from utils import get_file_paths


In [ ]:
root_path = './data'
dir_name = 'aso_data'
elev_tif = 'coge_dem_50m.tif'
swe_tif = 'ASO_50M_SWE_USCOGE_20180331.tif'
elev_path = os.path.join(root_path, dir_name,elev_tif)
swe_path = os.path.join(root_path, dir_name,swe_tif)

In [ ]:
def mask_values_in_range(array, min, max):
    z_mask = np.ma.masked_outside(array, min, max)

    return z_mask

In [ ]:
def values_in_zrange(values_array, values_bins, z_array, zmin, zmax):
    # the masked array with the elevation range input
    z_mask = mask_values_in_range(z_array, zmin, zmax)
    # the values within the elevation range mask provided from z_mask
    values_mask = np.ma.masked_array(values_array, ~z_mask.mask)

    values_1d = values_mask.data[values_mask.mask] # 1D vector of values not masked

    values_counts = np.zeros(values_1d.size)

    for i in np.arange(values_bins.size):
        values_counts[i] = np.count_nonzero(values_1d == values_bins[i])

    return values_counts

# Deal with the elevation raster

In [ ]:
with rio.open(elev_path) as elev:
    elev_array = elev.read(1).astype(float)

z_min = np.nanmin(elev_array)
z_max = np.nanmax(elev_array)
z_mean = np.nanmean(elev_array)
z_std = np.std(elev_array)
delta_z = 100 # arbitrary value to define elevation intervals

In [ ]:
def get_raster_stats(raster_path, stats_name, mask=False, unit=None):
    with rio.open(raster_path) as src:
        if mask:
            array = src.read(1,masked=mask).astype(float)
            array = array.filled(np.nan)
            array[array == 0] = np.nan
        else:
            array = src.read(1).astype(float)
        

    z_min = np.nanmin(array)
    z_max = np.nanmax(array)
    z_mean = np.nanmean(array)
    z_std = np.std(array)

    stats = f'''
    {stats_name} Statistics:
    Number of columns [east-west]: {array.shape[1]} 
    Number of rows [north-south]: {array.shape[0]}
    Minimum {stats_name}: {z_min:.{2}f} {unit}
    Maximum {stats_name}: {z_max:.{2}f} {unit}
    Mean {stats_name}: {z_mean:.{2}f} {unit}
    Std. dev. {stats_name}: {z_std:.{2}f} {unit}
    {stats_name} interval: {delta_z:.{2}f} {unit}
    '''

    print(stats)

    return src

In [ ]:
zstats = f'''
Elevation Statistics:
Number of columns [east-west]: {elev_array.shape[1]} 
Number of rows [north-south]: {elev_array.shape[0]}
Minimum elevation: {z_min:.{2}f} m
Maximum elevation: {z_max:.{2}f} m
Mean elevation: {z_mean:.{2}f} m
Std. dev. elevation: {z_std:.{2}f} m
elevation interval: {delta_z:.{2}f} m
'''

print(zstats)

In [ ]:
z_bot = np.floor(z_min/delta_z)*delta_z
z_top = np.ceil(z_max/delta_z)*delta_z

In [ ]:
z_intervals = np.arange(z_bot, z_top+delta_z, delta_z)
z_bin_center = (z_intervals[:-1] + z_intervals[1:]) / 2

In [ ]:
plt.imshow(elev_array)

# Deal with the swe raster

In [ ]:
# initialize the swe raster and deal with zero values
# check the spatial ref and units
with rio.open(swe_path) as swe:
    swe_array = swe.read(1, masked=True).astype(float)
    swe_array = swe_array.filled(np.nan) # set the zero values to nan
    swe_array[swe_array == 0] = np.nan
    swe_transform =swe.transform

swe_min = np.nanmin(swe_array)
swe_max = np.nanmax(swe_array)
swe_mean = np.nanmean(swe_array)
swe_std = np.nanstd(swe_array)

In [ ]:
swe_stats = f'''
SWE Statistics:
Number of columns [east-west]: {swe_array.shape[1]} 
Number of rows [north-south]: {swe_array.shape[0]}
Minimum SWE: {swe_min:.{2}f} mm
Maximum SWE: {swe_max:.{2}f} mm
Mean SWE: {swe_mean:.{2}f} mm
Standard dev. SWE: {swe_std:.{2}f} mm
elevation interval: {delta_z:.{2}f} m
'''

print(swe_stats)

In [ ]:
plt.imshow(swe_array)

In [ ]:
# calculate the area of a pixel in the swe raster
px_width = abs(swe_transform.a)
px_hieght = abs(swe_transform.e)
swe_cell = px_width * px_hieght

In [ ]:
# this work but I don't get it
def get_swe_stats_by_elev(elev_array, swe_array, z_intervals):
    data = {}
    data["n_bins"] = z_intervals.size - 1
    data['swe_mean_z'] = np.full(data['n_bins'], np.nan)
    data['swe_median_z'] = np.full(data['n_bins'], np.nan)
    data['swe_std_z'] = np.full(data['n_bins'], np.nan)
    data['swe_area_z'] = np.full(data['n_bins'], np.nan)
    data['swe_vol_z'] = np.full(data['n_bins'], np.nan)

    for i in range(data['n_bins']):
        bin_mask = (elev_array >= z_intervals[i]) & (elev_array < z_intervals[i + 1])
        # quick debug counters if you want
        # print(i, "pixels in bin:", np.count_nonzero(bin_mask))

        # select SWE values for this elevation band
        swe_bin = swe_array[bin_mask]

        # remove NaNs (nodata) from the selection
        if swe_bin.size == 0:
            # no pixels in elevation band -> leave as np.nan
            continue

        valid = swe_bin[~np.isnan(swe_bin)]
        if valid.size == 0:
            # all selected pixels are nodata -> leave as np.nan
            continue

        # compute statistics on valid values and assign into array
        data['swe_mean_z'][i] = valid.mean()        # or np.mean(valid)
        data['swe_median_z'][i] = np.median(valid)
        data['swe_std_z'][i] = valid.std()          # or np.std(valid)

        n_pixels = np.count_nonzero(~np.isnan(swe_bin))
        data['swe_area_z'][i] = n_pixels * swe_cell
        data['swe_vol_z'][i] = np.nansum(valid) * swe_cell * 1e-6

    return data


In [ ]:
swe_z = get_swe_stats_by_elev(elev_array, swe_array, z_intervals)

In [ ]:

def plot_swe(swe_z, z_bin_center):
    fig, ax = plt.subplots(ncols= 3, figsize=(10,8))
    ax[0].plot(swe_z['swe_mean_z'], z_bin_center, label='Mean Swe', color='k')
    ax[0].plot(swe_z['swe_median_z'], z_bin_center, label='Median Swe', color='b')

    ax[0].fill_betweenx(z_bin_center,
                        (swe_z['swe_mean_z'] - swe_z['swe_std_z']),
                        (swe_z['swe_mean_z'] + swe_z['swe_std_z']),
                        alpha=0.3, color='purple',label='1 std dev'
                        )
    ax[0].set_xlabel('SWE [mm]')
    ax[0].set_ylabel('Elevation')
    ax[0].grid(True, alpha=0.3)
    #ax[0].legend()

    # swe area plot
    ax[1].barh(z_bin_center, swe_z['swe_area_z'], height=delta_z, color='orange', edgecolor='orange', label='swe area by z')
    ax[1].set_xlabel('area [km2]')
    ax[2].barh(z_bin_center, swe_z['swe_vol_z'], height=delta_z, facecolor='purple', edgecolor='purple',label='swe volume by z')
    ax[2].set_xlabel('Volume [Mm3]')

    # make sure ax is iterable
    if isinstance(ax, Axes):
        axes = [ax]
    else:
        axes = np.array(ax).ravel()

    # apply grid + legend to each axes, but only call legend when there are handles
    for a in axes:
        a.grid(True, linestyle="--", linewidth=0.7, alpha=0.5)
        handles, labels = a.get_legend_handles_labels()
        if handles:
            a.legend(loc="lower right", fontsize="small")

    # optional: shared y-label and tidy layout
    fig.supylabel("Elevation (m)")
    fig.tight_layout()
    plt.show()

# Put the raster datasets together

In [ ]:
nz = z_intervals.size
nswe = swe_array.size # this probably isn't right
print(nz, nswe)

In [ ]:
# compute swe metrics (volume, area) per elevation band

swe_by_z_fracs = np.zeros((nz-1))
zfracs = np.zeros((nz-1))

for i in np.arange(nz-1):
    swe_dz = values_in_zrange(swe_array,z_intervals, elev_array, zmin=z_intervals[i], zmax=z_intervals[i+1])
    swe_area_dz = swe_dz.sum()*swe_cell # area of swe per z interval

    swe_by_z_fracs[i] = swe_area_dz



Horizontal Plots
1. Mean, and median swe on x-axis elevation on y-axis. Fill between of 1 std Dev.
2. horizontal bar plot of the cumulative area (pixel area * 1000^2) of swe pixels on x-axis for each elevation slice (50-100 m) on y-axis.
3. horizontal bar plot of the cumulative Volume (pixel area * swe value *10^6) of swe pixels on x-axis for each elevation slice (50-100 m) on y-axis.

In [ ]:

fig, ax = plt.subplots(figsize=(10,8))
left = 0.0

for i in np.arange(swe_bins.size):
    ax.barh(z_mean, swe_freq[i], left=left, height=500.0,label='test')
    left += swe_freq[i]

plt.show()
